In [37]:
def get_columns(file_path):
    with open(file_path) as f:
        column_line = f.readline()
        columns = column_line.split("=")[1]
        columns = [c.strip() for c in columns.split(" ") if c]
        return columns


In [38]:
get_columns("./data/train-devel-test/train/fiction/no-morph/chilcote_12.conllup")

['FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'CONLL:NER']

In [39]:
pip install conll-df

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 12.1 MB 3.5 MB/s            
     |████████████████████████████████| 500 kB 39.7 MB/s            
     |████████████████████████████████| 17.1 MB 55.7 MB/s            
Using legacy 'setup.py install' for conll-df, since package 'wheel' is not installed.
    Running setup.py install for conll-df ... done
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [94]:
def read_conllup_file(file_path):
    conllu_columns = get_columns("./data/train-devel-test/train/fiction/no-morph/chilcote_12.conllup")
    df = pd.DataFrame(columns=["sentence_index", "position_number_in_sentence", *conllu_columns])
    with open(file_path) as f:
        sentence_index = 0
        position_number_in_sentence = 0
        line = f.readline() # skip the first comment line
        line = f.readline()
        while line:
            if line == "\n":
                sentence_index +=  1
                position_number_in_sentence = 0
            else:
                splitted_line = line.split()
                conllu_row_part = {}
                for i in range(len(conllu_columns)):
                    conllu_row_part[f"{conllu_columns[i]}"] = splitted_line[i]
                
                row = {
                    "sentence_index": sentence_index,
                    "position_number_in_sentence": position_number_in_sentence,
                    **conllu_row_part
                }
                row = pd.Series(row)
                df.loc[len(df)] = row
                position_number_in_sentence += 1
            line = f.readline()
        return df

In [95]:
df = read_conllup_file("./data/train-devel-test/train/fiction/no-morph/chilcote_12.conllup")

In [105]:
df.head(40)

,sentence_index,position_number_in_sentence,FORM,LEMMA,UPOS,XPOS,FEATS,CONLL:NER
0,0,0,-,_,_,_,_,O
1,0,1,Korán,_,_,_,_,O
2,0,2,ebédelnek,_,_,_,_,O
3,0,3,",",_,_,_,_,O
4,0,4,úgy,_,_,_,_,O
5,0,5,",",_,_,_,_,O
6,0,6,hogy,_,_,_,_,O
7,0,7,most,_,_,_,_,O
8,0,8,már,_,_,_,_,O
9,0,9,öltözködnöm,_,_,_,_,O


In [108]:
df["CONLL:NER"].value_counts()

O         3637
B-PER       50
B-LOC       21
I-LOC       14
B-MISC       1
Name: CONLL:NER, dtype: int64